# Análisis exploratorio con datos de Netflix

En este cuaderno vamos a explorar la base de datos `netflix_titles.csv` usando **pandas** y **NumPy**.

Objetivos:
- Practicar carga y exploración de datos.
- Realizar limpieza y transformación básica.
- Responder preguntas usando agregaciones y filtros.
- Usar NumPy para calcular estadísticas sobre una variable numérica.


In [ ]:
import pandas as pd
import numpy as np


## 1. Carga e inspección inicial

En esta sección cargamos el archivo `netflix_titles.csv` y revisamos su estructura general.


In [ ]:
# TODO: ajusta la ruta si es necesario
df = pd.read_csv('netflix_titles.csv')
df.head()


In [ ]:
df.shape


In [ ]:
df.info()


In [ ]:
df.isna().sum()


In [ ]:
df.describe(include='all')


**Actividad:** Describe brevemente (en tus propias palabras) qué información contiene el dataset y qué columnas parecen más importantes para el análisis.


## 2. Limpieza y transformación básica

Vamos a:
- Convertir `date_added` a tipo fecha.
- Crear una columna `year_added`.
- Separar `duration` en número y unidad.
- Crear una columna de país principal.


In [ ]:
# Conversión de fecha
df['date_added'] = pd.to_datetime(df['date_added'])
df['year_added'] = df['date_added'].dt.year
df[['date_added', 'year_added']].head()


In [ ]:
# Separar duración en número y unidad
df['duration_num'] = df['duration'].str.extract(r'(\d+)').astype(float)
df['duration_unit'] = df['duration'].str.extract(r'([A-Za-z ]+)$').str.strip()
df[['duration', 'duration_num', 'duration_unit']].head()


In [ ]:
# País principal (primer país de la lista)
df['main_country'] = df['country'].str.split(',').str[0].str.strip()
df[['country', 'main_country']].head()


**Actividad:** Revisa qué columnas tienen muchos valores faltantes y comenta qué decisión tomarías (eliminar filas, imputar, dejar como están) para este proyecto.


## 3. Preguntas de análisis con pandas

Responde las siguientes preguntas usando filtros, `groupby` y agregaciones.


### 3.1. Películas vs series

¿Cuántas películas y cuántas series hay en el catálogo?


In [ ]:
df['type'].value_counts()


### 3.2. Títulos añadidos por año y tipo

¿Cómo ha evolucionado el número de títulos añadidos a Netflix por año (`year_added`) y por `type`?


In [ ]:
titles_per_year = (
    df.groupby(['year_added', 'type'])['show_id']
      .count()
      .reset_index(name='n_titles')
)
titles_per_year.head()


### 3.3. Duración de las películas por década

Considera solo los registros donde `type == 'Movie'`. Calcula la duración promedio por década de `release_year`.


In [ ]:
movies = df[df['type'] == 'Movie'].copy()
movies['decade'] = (movies['release_year'] // 10) * 10
duration_by_decade = (
    movies.groupby('decade')['duration_num']
          .mean()
)
duration_by_decade


### 3.4. Géneros más frecuentes

¿Cuáles son los géneros (`listed_in`) más comunes en el catálogo?


In [ ]:
genres_series = df['listed_in'].dropna().str.split(', ')
all_genres = pd.Series([g for sub in genres_series for g in sub])
top_genres = all_genres.value_counts().head(10)
top_genres


### 3.5. Países con más títulos

Usando `main_country`, ¿cuáles son los países con más títulos en el catálogo?


In [ ]:
df['main_country'].value_counts().head(10)


## 4. Uso de NumPy para estadísticas

En esta sección usamos NumPy explícitamente sobre la duración de las películas.


In [ ]:
movie_durations = movies['duration_num'].dropna().values
movie_durations[:10]


In [ ]:
mean_duration = np.mean(movie_durations)
median_duration = np.median(movie_durations)
p90_duration = np.percentile(movie_durations, 90)
mean_duration, median_duration, p90_duration


In [ ]:
long_mask = movie_durations > 120
long_ratio = long_mask.mean()
long_ratio


**Actividad:** Interpreta los resultados anteriores: ¿es común encontrar películas muy largas (> 120 minutos) en el catálogo?


## 5. Conclusiones

Escribe un breve resumen (5–8 frases) donde respondas, en lenguaje natural, a las preguntas principales del análisis:
- Películas vs series.
- Evolución por año.
- Patrón de duración por década.
- Géneros y países predominantes.
